In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install swifter
!pip install dgl-cu100
!pip install pytorch-metric-learning

In [0]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import time
import gc
from glob import glob
from tqdm.notebook import tqdm
import pickle
import logging
import json

import base64
import swifter
import gensim

from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset

pd.set_option('display.max_columns', 500)

In [4]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

import keras
from keras.preprocessing.sequence import pad_sequences

import networkx as nx
import dgl
import dgl.function as fn
from dgl.nn.pytorch import GraphConv

from pytorch_metric_learning import losses

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,DistilBertTokenizer,DistilBertModel
)

print(transformers.__version__)

Using TensorFlow backend.
Using backend: pytorch
ERROR:root:The testing module requires faiss. You can install the GPU version with the command 'conda install faiss-gpu -c pytorch' 
                        or the CPU version with 'conda install faiss-cpu -c pytorch'. Learn more at https://github.com/facebookresearch/faiss/blob/master/INSTALL.md


2.9.0


In [5]:
os.getcwd()

'/content'

In [0]:
DATA_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/data/'
LOG_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/log/'
W2V_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/w2v/'
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/model/'
SUBMIT_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/prediction_result/'

In [0]:
# DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/data/'
# LOG_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/log/'
# W2V_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/w2v/'
# BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT/'
# MODEL_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/model/'

### Read Data

In [8]:
train = pd.read_csv(DATA_DIR+'train/train.sample.tsv', sep='\t')
print(train.shape)
train.head()

(10000, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,102851856,338,209,2,AAAAAAAAQEAAAKlDAABPQwAAb0MAALBBAICmQwAATEM=,AAAAAAAAAAAnNsFAAAAAAOeVGD0AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAAA==,treble popular reed,923271
1,101783080,80,61,1,AACgQAAAoEAAAJZCAABgQg==,AAAAADnKfjyatb8/AAAAALxPBz0MH6E9xwkoPAAAAAAAAA...,HAAAAAAAAAA=,compatible ink cartridges,234031
2,100492797,750,750,1,AAAsQgCAg0MAgCREAADqQw==,AAAAAAAAAABmUlE/AAAAABvc0jwAAAAAAAAAAAAAAAAAAA...,BwAAAAAAAAA=,check new look facial cream,199926
3,101708088,480,640,5,AAB6QwCA0kMAANBDAMAeRAAAfEMAAKBAAIDWQwAAVkMAAH...,AAAAAAAAAAAAAAAAAAAAAAXyfz4AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAABwAAAAAAAAAHAAAAAAAAAAcAA...,royal commemorative stamp,772725
4,100316842,800,800,4,AAAKQwCAgEMAADNEAAAHRAAAIkMAwCZEAACqQwDARkQAAE...,AAAAADSdaTwVTdA8AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAGAAAAAAAAAAYAAAAAAAAAHAAAAAAAAAA=,calendula baby shower gel,362576


In [9]:
valid = pd.read_csv(DATA_DIR+'valid/valid.tsv', sep='\t')
print(valid.shape)
valid.head()

(14720, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,103047314,800,800,2,AACoQgAA5EIAAKdDAIDuQwCAq0MAAJBCAMA3RABAHkQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
1,103030678,800,800,9,AIDRQwAAoEAAwEJEAABpQwAAA0MAgMlDAABHRAAAIkQAgL...,AAAAAAAAAACdFBI7AAAAAAAAAABu+j0+AAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAAQAAAAAAAAAHQAAAAAAAAAEAA...,breathable and comfortable children's shoes,0
2,103031217,488,750,2,AABGQwAANEIAgNNDAADsQwAAUEIAALVDAADRQwCALEQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
3,103026621,929,800,10,AACtQwCAA0QAgPxDAIAoRAAAekMAALdDAACwQwCA30MAgK...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAdAAAAAAAAAAAAAAAAAAAACQAAAAAAAAAJAA...,breathable and comfortable children's shoes,0
4,103017048,800,800,2,AAA+QwCAi0MAgAtEAMA0RAAAqEMAAHhCAMA1RABADUQ=,AAAAAAAAAADIb2s9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0


In [10]:
testA = pd.read_csv(DATA_DIR+'testA/testA.tsv', sep='\t')
print(testA.shape)
testA.head()

(28830, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,103052445,798,798,1,AIC3QwAAF0MAABxEAMAtRA==,AAAAAAAAAABwKQE9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0
1,103044513,800,800,1,AABqQwAAzEIAADJEAMAgRA==,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0
2,103035030,800,800,8,AAAAQwAAAEEAgNRDAICGQwAAUEEAAPBBAACUQgAA3EIAAI...,AAAAAAAAAABFkBg+AAAAAM5daTwAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAFAAAAAAAAAAIAAAAAAAAAAwAAAAAAAAAEAA...,straw heavy-bottomed sandals,0
3,103036197,800,800,3,AAAmQwAAwEEAwC1EAICqQwAAJ0MAAFJDAAA1RADAAkQAAE...,AAAAAAAAAAD2rhE+AAAAAAAAAADbLAw+AAAAANGzLD4AAA...,BAAAAAAAAAAEAAAAAAAAAAQAAAAAAAAA,straw heavy-bottomed sandals,0
4,103055323,800,800,1,AABUQwAA+EEAgChEAEBCRA==,AAAAAKzRGTs0osc+kJpVPAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0


### Decode Data

In [0]:
def transform_boxes(df_row):
    return np.frombuffer(base64.b64decode(df_row.boxes), dtype=np.float32).reshape(df_row.num_boxes, 4)

def transform_features(df_row):
    return np.frombuffer(base64.b64decode(df_row.features), dtype=np.float32).reshape(df_row.num_boxes, 2048)

def transform_class_labels(df_row):
    return np.frombuffer(base64.b64decode(df_row.class_labels), dtype=np.int64).reshape(df_row.num_boxes)

def transform_df(df):
    df.boxes = df.swifter.apply(transform_boxes, axis=1)
    df.features = df.swifter.apply(transform_features, axis=1)
    df.class_labels = df.swifter.apply(transform_class_labels, axis=1)
    return df

In [12]:
train = transform_df(train)
valid = transform_df(valid)
testA = transform_df(testA)

In [13]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[28, 28]",treble popular reed,923271
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[28],compatible ink cartridges,234031
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...",[7],check new look facial cream,199926
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[28, 28, 28, 28, 28]",royal commemorative stamp,772725
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[28, 6, 6, 28]",calendula baby shower gel,362576


### Feature Engineering

- box area

In [14]:
train['box_normalized'] = train.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)
valid['box_normalized'] = valid.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)
testA['box_normalized'] = testA.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)

In [15]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_normalized
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[28, 28]",treble popular reed,923271,"[[0.0, 0.014354066985645933, 1.0, 0.9904306220..."
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[28],compatible ink cartridges,234031,"[[0.0625, 0.08196721311475409, 0.9375, 0.91803..."
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...",[7],check new look facial cream,199926,"[[0.05733333333333333, 0.3506666666666667, 0.8..."
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[28, 28, 28, 28, 28]",royal commemorative stamp,772725,"[[0.5208333333333334, 0.6578125, 0.86666666666..."
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[28, 6, 6, 28]",calendula baby shower gel,362576,"[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8..."


In [0]:
# box area
train['box_area_normalized'] = train.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))
valid['box_area_normalized'] = valid.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))
testA['box_area_normalized'] = testA.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))

In [17]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_normalized,box_area_normalized
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[28, 28]",treble popular reed,923271,"[[0.0, 0.014354066985645933, 1.0, 0.9904306220...","[0.9760765550239234, 0.2421788737578212]"
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[28],compatible ink cartridges,234031,"[[0.0625, 0.08196721311475409, 0.9375, 0.91803...",[0.7315573770491803]
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...",[7],check new look facial cream,199926,"[[0.05733333333333333, 0.3506666666666667, 0.8...",[0.2241333333333333]
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[28, 28, 28, 28, 28]",royal commemorative stamp,772725,"[[0.5208333333333334, 0.6578125, 0.86666666666...","[0.11563802083333333, 0.120419921875, 0.133255..."
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[28, 6, 6, 28]",calendula baby shower gel,362576,"[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8...","[0.25558437500000003, 0.0356, 0.03279375, 0.21..."


- word2vec of box labels

In [18]:
class_label_dict = {
    0:'top clothes (coat, jacket, shirt, etc.)',
    1:'skirt & dress',
    2:'bottom clothes (trousers, pants, etc.)',
    3:'luggage, leather goods',
    4:'shoes',
    5:'accessories (jewelry, clothing accessories, belts, hats, scarves, etc.)',
    6:'snacks, nuts, liquor and tea',
    7:'makeup, perfume, beauty tools and essential oils',
    8:'bottle drink',
    9:'furniture',
    10:'stationery',
    11:'household electrical appliances',
    12:'home decoration',
    13:'household fabric',
    14:'kitchenware',
    15:'home / personal cleaning tools',
    16:'storage supplies',
    17:'motorcycle, motorcycle accessories, vehicles, bicycle and riding equipment',
    18:'outdoor product',
    19:'lighting',
    20:'toys',
    21:'underwear',
    22:'digital supplies',
    23:'bed linens',
    24:'baby products',
    25:'personal care',
    26:'sporting goods',
    27:'clothes (accessories, baby clothing, etc.)',
    28:'others',
    29:'human face',
    30:'arm',
    31:'hair',
    32:'hand',
}

train.class_labels = train.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
valid.class_labels = valid.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
testA.class_labels = testA.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))

In [0]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path, encoding='utf-8') as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():  # word_indexのwordに対応するembがあれば、embを代入する
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words


def load_word_embed(word_embed_glove=W2V_DIR+"glove.840B.300d.txt", 
                    word_embed_crawl=W2V_DIR+"crawl-300d-2M.vec",
                    save_filename=W2V_DIR+"word_embedding_matrix_testA",
                    word_index=None):
    
    # Tokneizerの学習
    Tokenizer = keras.preprocessing.text.Tokenizer(filters='', lower=False)
    Tokenizer.fit_on_texts(list(train['query'])+list(valid['query'])+list(testA['query'])
                           +list(train['class_labels'])+list(valid['class_labels'])+list(testA['class_labels']))
    
    if os.path.exists(save_filename + ".npy"):
        embedding_matrix = np.load(save_filename + ".npy").astype("float32")
    else:
        
        if word_index is None:
            word_index = Tokenizer.word_index
        
        glove_matrix, unknown_words_glove = build_matrix(word_index, word_embed_glove)
        print('n unknown words (glove): ', len(unknown_words_glove))
        
        crawl_matrix, unknown_words_crawl = build_matrix(word_index, word_embed_crawl)
        print('n unknown words (crawl): ', len(unknown_words_crawl))
        
        embedding_matrix = crawl_matrix + glove_matrix  
        np.save(save_filename, embedding_matrix)

        del crawl_matrix
        del glove_matrix
        gc.collect()
        
    return embedding_matrix, Tokenizer


def calc_mean_w2v(class_labels):
    class_labels_index = tokenizer_w2v.texts_to_sequences([class_labels])
    return np.mean(embedding_matrix[class_labels_index], axis=0)
    
    # try:
    #     return embedding_matrix[class_labels_index]
    #     #return np.mean(embedding_matrix[class_labels_index], axis=0)
    # except KeyError:
    #     print('Error')
    #     return np.zeros(300)

In [0]:
# about 5 minites
embedding_matrix, tokenizer_w2v = load_word_embed()

In [0]:
# train['class_labels_vec'] = train['class_labels'].swifter.apply(calc_mean_w2v)
# valid['class_labels_vec'] = valid['class_labels'].swifter.apply(calc_mean_w2v)
# testA['class_labels_vec'] = testA['class_labels'].swifter.apply(calc_mean_w2v)

- Adjaccency matrix
  - 面積が一番大きいboxを中心に、それと他の全ての結合グラフを考える

In [0]:
# from numba import jit

# @jit
# def get_adj_mat(X, Y, adj_mat):
#     n_box = adj_mat.shape[0]
#     if n_box!=1:
#         for i in range(n_box):
#             for j in range(i+1, n_box):
#                 d = np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2)
#                 adj_mat[i, j] = 1/d
#                 adj_mat[j, i] = 1/d
                
#     return adj_mat

# def get_adj_mat_row(x):
#     n_box = x.num_boxes
#     adj_mat = np.zeros((n_box, n_box))
#     return get_adj_mat(x.center_normalized[:,0], x.center_normalized[:,1], adj_mat)

In [0]:
# top/left/bottom/right
train['center_normalized'] = train.box_normalized.apply(lambda x: np.array([[((xi[3]-xi[1])/2+xi[1]), ((xi[2]-xi[0])/2+xi[0])] for xi in x]))
valid['center_normalized'] = valid.box_normalized.apply(lambda x: np.array([[((xi[3]-xi[1])/2+xi[1]), ((xi[2]-xi[0])/2+xi[0])] for xi in x]))
testA['center_normalized'] = testA.box_normalized.apply(lambda x: np.array([[((xi[3]-xi[1])/2+xi[1]), ((xi[2]-xi[0])/2+xi[0])] for xi in x]))

In [24]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_normalized,box_area_normalized,center_normalized
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...",others others,treble popular reed,923271,"[[0.0, 0.014354066985645933, 1.0, 0.9904306220...","[0.9760765550239234, 0.2421788737578212]","[[0.5023923444976076, 0.5], [0.540669856459330..."
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",others,compatible ink cartridges,234031,"[[0.0625, 0.08196721311475409, 0.9375, 0.91803...",[0.7315573770491803],"[[0.5, 0.5]]"
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...","makeup, perfume, beauty tools and essential oils",check new look facial cream,199926,"[[0.05733333333333333, 0.3506666666666667, 0.8...",[0.2241333333333333],"[[0.48733333333333334, 0.4673333333333333]]"
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....",others others others others others,royal commemorative stamp,772725,"[[0.5208333333333334, 0.6578125, 0.86666666666...","[0.11563802083333333, 0.120419921875, 0.133255...","[[0.825, 0.6937500000000001], [0.17109375, 0.7..."
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","others snacks, nuts, liquor and tea snacks, nu...",calendula baby shower gel,362576,"[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8...","[0.25558437500000003, 0.0356, 0.03279375, 0.21...","[[0.49812500000000004, 0.53375], [0.9137500000..."


In [0]:
# train['adj_mat'] = train.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)
# valid['adj_mat'] = valid.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)
# testA['adj_mat'] = testA.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)

In [26]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_normalized,box_area_normalized,center_normalized
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...",others others,treble popular reed,923271,"[[0.0, 0.014354066985645933, 1.0, 0.9904306220...","[0.9760765550239234, 0.2421788737578212]","[[0.5023923444976076, 0.5], [0.540669856459330..."
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",others,compatible ink cartridges,234031,"[[0.0625, 0.08196721311475409, 0.9375, 0.91803...",[0.7315573770491803],"[[0.5, 0.5]]"
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...","makeup, perfume, beauty tools and essential oils",check new look facial cream,199926,"[[0.05733333333333333, 0.3506666666666667, 0.8...",[0.2241333333333333],"[[0.48733333333333334, 0.4673333333333333]]"
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....",others others others others others,royal commemorative stamp,772725,"[[0.5208333333333334, 0.6578125, 0.86666666666...","[0.11563802083333333, 0.120419921875, 0.133255...","[[0.825, 0.6937500000000001], [0.17109375, 0.7..."
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","others snacks, nuts, liquor and tea snacks, nu...",calendula baby shower gel,362576,"[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8...","[0.25558437500000003, 0.0356, 0.03279375, 0.21...","[[0.49812500000000004, 0.53375], [0.9137500000..."


### Negative Sample生成
- ここでは、簡単のため、queryだけランダムにシャッフルしたデータフレームを作成し、concatする
- ラベルとして、元データはlabel=1, ランダムシャッフルはlabel=0とする

In [0]:
# train['label'] = 1

# random.seed(42)
# ind = list(train.index)
# random.shuffle(ind)

# train_neg = train.copy(deep=True)
# train_neg['label'] = 0
# train_neg['query'] = list(train_neg['query'].loc[ind])[:]
# train_neg.head()

# train = pd.concat([train, train_neg], axis=0)
# train = train.reset_index(drop=True)
# print(train.shape)

In [28]:
train.iloc[0]

product_id                                                     102851856
image_h                                                              338
image_w                                                              209
num_boxes                                                              2
boxes                  [[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...
features               [[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...
class_labels                                               others others
query                                                treble popular reed
query_id                                                          923271
box_normalized         [[0.0, 0.014354066985645933, 1.0, 0.9904306220...
box_area_normalized             [0.9760765550239234, 0.2421788737578212]
center_normalized      [[0.5023923444976076, 0.5], [0.540669856459330...
Name: 0, dtype: object

## Model

- initial setting

In [0]:
# make logger
logger = logging.getLogger('main')
logger.setLevel(logging.DEBUG)
sc = logging.StreamHandler()
logger.addHandler(sc)
fh = logging.FileHandler(LOG_DIR+'20200505.log')
logger.addHandler(fh)

In [0]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [32]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


### DataLoader

In [0]:
MAX_LEN = 20
MAX_LEN_IMG = 1

class KDDDataset(Dataset):

    def __init__(self, df, train_mode=True, transform=None):
        self.df = df
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.w2v_tokenizer = tokenizer_w2v
        #self.transform = transform
        self.train_mode = train_mode
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        ######################
        # query side
        ######################
        query = row['query']
        # inputs_query = self.bert_tokenizer.encode_plus(
        #     query,
        #     add_special_tokens=True,
        #     max_length=MAX_LEN,
        # )
        
        # ids_query = inputs_query["input_ids"]
        # token_type_ids_query = inputs_query["token_type_ids"]
        # mask_query = inputs_query["attention_mask"]
        
        # padding_len = MAX_LEN - len(ids_query)
        # ids_query = ids_query + ([0]*padding_len)
        # token_type_ids_query = token_type_ids_query + ([0]*padding_len)
        # mask_query = mask_query + ([0]*padding_len)

        query = tokenizer_w2v.texts_to_sequences([query])
        query = pad_sequences(query, maxlen=MAX_LEN, padding='post')

        
        
        ######################
        # image side
        ######################
        # adj_mat = row['adj_mat']
        # features = row['features']
        # box_area = row['box_area_normalized']

        # box_areaの大きい順にソート
        box_order = np.argsort(row.box_area_normalized)[::-1]
        faetures_ordered = row.features[box_order, :][:MAX_LEN_IMG]
        box_ordered = row.box_normalized[box_order, :][:MAX_LEN_IMG]
        area_ordered = row.box_area_normalized[box_order][:MAX_LEN_IMG]
        #class_labels_ordered = row.class_labels[box_order][:MAX_LEN_IMG]
        
        features = pad_sequences([faetures_ordered.reshape(-1,)], maxlen=MAX_LEN_IMG*2048, padding='post')
        box = pad_sequences([box_ordered.reshape(-1,)], maxlen=MAX_LEN_IMG*4, padding='post')
        area = pad_sequences([area_ordered.reshape(-1,)], maxlen=MAX_LEN_IMG, padding='post')
                
        
        if self.train_mode:
            # positive, negative label
            #labels = row['label']
            return {
                'query': torch.tensor(query, dtype=torch.long),
                #'class_labels_vec': torch.tensor(class_labels_vec, dtype=torch.float),
                #'label': torch.tensor(labels, dtype=torch.float),
                #'label': torch.tensor(labels, dtype=torch.float),
                'features': torch.tensor(features, dtype=torch.float),
                'box': torch.tensor(box, dtype=torch.float),
                'area': torch.tensor(area, dtype=torch.float),
            }
        else:
            return {
                'query': torch.tensor(query, dtype=torch.long),
                'features': torch.tensor(features, dtype=torch.float),
                'box': torch.tensor(box, dtype=torch.float),
                'area': torch.tensor(area, dtype=torch.float),
            }

In [0]:
# def collate(samples):
#     # input samples は、ペアのリスト
#     #  [(graph, {features}),...], バッチサイズの長さのリスト
#     # グラフは、隣接行列を対角につなげて1つのバッチにする
#     # 他のfeatureは、(バッチサイズ, 次元数)のtensor
    
#     graphs, features_dict = map(list, zip(*samples))
    
#     batched_graph = dgl.batch(graphs)
    
#     batched_features = {}
#     labels = []
#     for i, feat in enumerate(features_dict):
#         if i==0:
#             for (key, val) in feat.items():
#                 if key!='label':
#                     batched_features[key] = val.view(1,-1)
#                 else:
#                     labels.append(val.item())
#         else:
#             for (key, val) in feat.items():
#                 if key!='label':
#                     batched_features[key] = torch.cat([batched_features[key],  val.view(1,-1)], dim=0)
#                 else:
#                     labels.append(val.item())
    
#     #batched_features['label'] = torch.tensor(labels, dtype=torch.long)
    
#     return batched_graph, batched_features


# def collate_test(samples):
#     # input samples は、ペアのリスト
#     #  [(graph, {features}),...], バッチサイズの長さのリスト
#     # グラフは、隣接行列を対角につなげて1つのバッチにする
#     # 他のfeatureは、(バッチサイズ, 次元数)のtensor
    
#     graphs, features_dict = map(list, zip(*samples))
    
#     batched_graph = dgl.batch(graphs)
    
#     batched_features = {}
#     labels = []
#     for i, feat in enumerate(features_dict):
#         if i==0:
#             for (key, val) in feat.items():
#                 if key!='label':
#                     batched_features[key] = val.view(1,-1)
#                 else:
#                     labels.append(val.item())
#         else:
#             for (key, val) in feat.items():
#                 if key!='label':
#                     batched_features[key] = torch.cat([batched_features[key],  val.view(1,-1)], dim=0)
#                 else:
#                     labels.append(val.item())
        
#     return batched_graph, batched_features

In [0]:
# # dataloaderの挙動確認

# dataset_train = KDDDataset(train, train_mode=True)
# train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)

# for i, (graph, features) in enumerate(train_loader):
#     if i<1:
#         print(graph.batch_size)
#         print(graph)
#         print(features['ids_query'].shape)
#         print(features['mask_query'].shape)
#         print(features['token_type_ids_query'].shape)
#         print(features['class_labels_vec'].shape)
#         print(features['label'].shape)

#     else:
#         break

## Model

In [0]:
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.fc = nn.Linear(in_feats, out_feats)
        self.activation = activation
    
    def message_func(self, edges):
        """edgeを通して，どのようなメッセージをどう渡すかを決める関数"""
        weight = edges.data['weight'].view(-1, 1)
        messages = edges.src['h'] * weight
        return {'m': messages}
    
    def reduce_func(self, nodes):
        """渡されたメッセージをどのように集約するかを決める関数"""
        messages = nodes.mailbox['m']
        h = torch.sum(messages, dim=1)
        return {'h': h}
    
    def node_apply_func(self, nodes):
        """ノードごとに作用させる関数"""
        h = self.fc(nodes.data['h'])
        h = self.activation(h)
        return {'h': h}

    def forward(self, g, h):
        g.ndata['h'] = h
        g.apply_nodes(self.node_apply_func)
        g.update_all(self.message_func, self.reduce_func)
        return g.ndata.pop('h')  # ndata['h']が削除されるが，返り値としては，ndata['h']となる

In [0]:
class QueryModel(nn.Module):
    def __init__(self):
        super(QueryModel, self).__init__()
        self.model_name = 'QueryModel'

        # query
        # self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        # self.fc_bert = nn.Linear(768, 128)

        self.max_feature = embedding_matrix.shape[0]
        self.embedding_size = embedding_matrix.shape[1]
      
        self.embedding = nn.Embedding(self.max_feature, self.embedding_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.required_grad = False

        # self.fc_emb = nn.Linear(300, 128)


    #def forward(self, ids, token_type_ids, mask):
    def forward(self, query):  
        ##############################
        # query
        ##############################
        # layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=token_type_ids, attention_mask=mask)
        # out_query = F.avg_pool1d(layers.transpose(1,2), kernel_size=layers.size()[1]).squeeze()
        # out_query = F.dropout(out_query, p=0.3, training=self.training)
        # out_query = F.relu(self.fc_bert(out_query))
        
        # print(query.size())
        out_query = self.embedding(query).squeeze()
        # print(out_query.size())
        out_query = F.avg_pool1d(out_query.transpose(1,2), kernel_size=out_query.size()[1]).squeeze()
        # out_query = F.relu(self.fc_emb(out_query))

        return out_query

In [0]:
# class ImageModel(nn.Module):
#     def __init__(self):
#         super(ImageModel, self).__init__()
#         self.model_name = 'ImageModel'

#         # image
#         self.gcn1 = GCN(2048+4+1, 512, F.relu)
#         self.gcn2 = GCN(512, 256, F.relu)
#         self.fc1 = nn.Linear(256, 128)
#         #self.fc2 = nn.Linear(128, 64)


#     def forward(self, g):
        
#         ##############################
#         # image
#         ##############################
#         h = g.ndata.pop('h')
#         h = self.gcn1(g, h)
#         h = self.gcn2(g, h)
#         g.ndata['h'] = h
        
#         for i, graph in enumerate(dgl.unbatch(g)):
#             if i==0:
#                 out_image = graph.ndata['h'][0].unsqueeze(0)
#             else:
#                 out_image = torch.cat([out_image, graph.ndata['h'][0].unsqueeze(0)], dim=0)


#         # Calculate graph representation by averaging all the node representations.
#         #out_image = dgl.mean_nodes(g, 'h', 'w')  # wでの重み付き平均
#         # out_image = dgl.max_nodes(g, 'h')
        
#         #out_image = torch.cat([out_image, class_labels_vec], dim=-1)
        
#         out_image = F.dropout(out_image, p=0.2, training=self.training)
#         out_image = F.relu(self.fc1(out_image))
#         #out_image = F.relu(self.fc2(out_image))
        
#         return out_image

In [0]:
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.model_name = 'ImageModel'

        # image
        # self.gcn1 = GCN(2048, 512, F.relu)
        # self.gcn2 = GCN(512, 256, F.relu)
        # self.fc1 = nn.Linear(256+300, 256)
        # self.fc2 = nn.Linear(256, 128)

        self.fc = nn.Linear((2048+5)*MAX_LEN_IMG, 512)
        self.fc2 = nn.Linear(512, 300)

    def forward(self, features, box, area):
        
        features = torch.cat([features, box, area], dim=-1)
        out_image = F.relu(self.fc(features)).squeeze()
        out_image = F.relu(self.fc2(out_image))
        
        return out_image

In [0]:
class KDDModel(nn.Module):
    def __init__(self):
        super(KDDModel, self).__init__()
        self.model_name = 'KDDModel'

        self.model_query = QueryModel()
        self.model_image = ImageModel()

        self.fc1 = nn.Linear(300+300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)


    def forward(self, query, features, box, area):
        
        out_query = self.model_query(query)
        out_image = self.model_image(features, box, area)

        out = torch.cat([out_query, out_image], dim=-1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        
        return out.squeeze()

### Training

In [0]:
def train_model(train_loader, model, optimizer, criterion): #, scheduler):
    model.train()
    avg_loss = 0.
    avg_score = 0.
    for idx, batch in enumerate(tqdm(train_loader)):
        batch_size = batch['query'].shape[0]
        
        idx_pos = random.sample(range(batch_size), k=batch_size//2)
        rest_set = list(set(range(batch_size)) - set(idx_pos))
        idx_neg1 = random.choices(rest_set, k=batch_size//2)
        idx_neg2 = random.choices(rest_set, k=batch_size//2)
        # idx_neg2 = idx_neg1[:]
        # random.shuffle(idx_neg2)

        query = torch.cat([batch['query'][idx_pos, :], batch['query'][idx_neg1, :]], dim=0).to(device)
        
        features = torch.cat([batch['features'][idx_pos, :], batch['features'][idx_neg2, :]]).to(device)
        box = torch.cat([batch['box'][idx_pos, :], batch['box'][idx_neg2, :]], dim=0).to(device)
        area = torch.cat([batch['area'][idx_pos, :], batch['area'][idx_neg2, :]], dim=0).to(device)
        
        labels = torch.cat([torch.tensor(np.ones(batch_size//2), dtype=torch.long), torch.tensor(np.zeros(batch_size//2), dtype=torch.long)], dim=0).to(device)


        out = model(query, features, box, area)
        loss = criterion(out, labels)

        _, predicted = torch.max(out, 1)
        correct = (predicted == labels).sum().item()
        acc = float(correct) / labels.size(0)
        print('acc', acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item() / len(train_loader)
        avg_score += acc / len(train_loader)

        del query,out,loss

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss, avg_score



def test_model(test_loader, model):
    model.eval()
    
    out_list = []   
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(test_loader)):
            query = batch['query'].to(device)
            features = batch['features'].to(device)
            box = batch['box'].to(device)
            area = batch['area'].to(device)

            out = F.softmax(model(query, features, box, area))[:,1]
            # out_query = model.model_query(ids_query, token_type_ids_query, mask_query)
            # out_image = model.model_image(graph)
            # out = F.cosine_similarity(out_query, out_image)

            out_list.append(out)
            del query,out
        
    torch.cuda.empty_cache()
    gc.collect()
    return out_list

In [0]:
# compute dcg@k for a single sample
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(3, r.size + 2)))
    return 0.


# compute ndcg@k (dcg@k / idcg@k) for a single sample
def get_ndcg(r, ref, k):
    dcg_max = dcg_at_k(ref, k)
    if not dcg_max:
        return 0.
    dcg = dcg_at_k(r, k)
    return dcg / dcg_max


def make_score(df_test, out_list):
    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    predictions = {}
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        predictions[f'{q_id}'] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values.astype(str)
    
    
    # read ground-truth
    reference = json.load(open(DATA_DIR+'valid/valid_answer.json'))

    # compute score for each query
    k = 5
    score_sum = 0.
    for qid in reference.keys():
        ground_truth_ids = set([str(pid) for pid in reference[qid]])
        ref_vec = [1.0] * len(ground_truth_ids)
        pred_vec = [1.0 if pid in ground_truth_ids else 0.0 for pid in predictions[qid]]
        score_sum += get_ndcg(pred_vec, ref_vec, k)
    # the higher score, the better
    score = score_sum / len(reference)
    
    return score

In [48]:
model = KDDModel().to(device)

# if device == 'cuda':
#     model = torch.nn.DataParallel(model) # make parallel
#     cudnn.benchmark = True


optimizer = torch.optim.Adam(model.parameters())

# optimizer = torch.optim.Adam([
#                         {'params': model.model_query.parameters()},
#                         {'params': model.model_image.parameters()}
#                     ])


criterion = nn.CrossEntropyLoss()
#criterion = nn.CosineSimilarity()
# criterion = nn.MSELoss()
#criterion = nn.CosineEmbeddingLoss()
#criterion = losses.TripletMarginLoss(margin=0.1)

loss_list_epoch_train = []
score_list_epoch_valid = []
score_best = 0
patience = 0
EPOCHS = 10
for epoch in range(EPOCHS):

    torch.cuda.empty_cache()
    start_time   = time.time()

    dataset_train = KDDDataset(train, train_mode=True)
    dataset_valid = KDDDataset(valid, train_mode=False)
    train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True)
    valid_loader = DataLoader(dataset_valid, batch_size=128, shuffle=False, num_workers=0, drop_last=False)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: 0.95 ** epoch)
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=EPOCHS)

    loss_train, score_train = train_model(train_loader, model, optimizer, criterion) #, scheduler)
    #scheduler.step()
    out_valid = test_model(valid_loader, model)
    score_valid = make_score(valid, out_valid)

    loss_list_epoch_train.append(loss_train)
    score_list_epoch_valid.append(score_valid)

    logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, train_acc: {score_train}, valid_score: {score_valid}, time: {(time.time()-start_time)}')

    # Eearly Stopping
    if score_valid > score_best:
        score_best = score_valid
        best_param = model.state_dict()
        torch.save(best_param, MODEL_DIR+f'best_param.pt')
        patience = 0
    else:
        patience += 1
        if patience > 2:
            del train_loader, valid_loader, loss_train, out_valid
            torch.cuda.empty_cache()
            gc.collect()
            break

    del train_loader, valid_loader, loss_train, out_valid
    torch.cuda.empty_cache()
    gc.collect()

acc 0.5
acc 0.5
acc 0.4921875
acc 0.5078125
acc 0.4453125
acc 0.5703125
acc 0.4921875
acc 0.515625
acc 0.6015625
acc 0.515625
acc 0.5
acc 0.546875
acc 0.5546875
acc 0.4765625
acc 0.5390625
acc 0.5234375
acc 0.546875
acc 0.5078125
acc 0.53125
acc 0.5390625
acc 0.546875
acc 0.4296875
acc 0.625
acc 0.5234375
acc 0.5234375
acc 0.4296875
acc 0.5078125
acc 0.515625
acc 0.53125
acc 0.5390625
acc 0.5625
acc 0.515625
acc 0.59375
acc 0.453125
acc 0.4765625
acc 0.5625
acc 0.53125
acc 0.5078125
acc 0.640625
acc 0.53125
acc 0.59375
acc 0.59375
acc 0.5390625
acc 0.5390625
acc 0.609375
acc 0.609375
acc 0.6015625
acc 0.6640625
acc 0.6171875
acc 0.6484375
acc 0.6328125
acc 0.59375
acc 0.6171875
acc 0.625
acc 0.59375
acc 0.6484375
acc 0.640625
acc 0.5703125
acc 0.671875
acc 0.6796875
acc 0.7109375
acc 0.640625
acc 0.640625
acc 0.578125
acc 0.6171875
acc 0.609375
acc 0.6640625
acc 0.7421875
acc 0.671875
acc 0.5859375
acc 0.6640625
acc 0.6328125
acc 0.71875
acc 0.65625
acc 0.6796875
acc 0.6328125
acc 0.68

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1, train_loss: 0.683712555047793, train_acc: 0.5765224358974357, valid_score: 0.28144144411065625, time: 17.418983936309814
INFO:main:Epoch 1, train_loss: 0.683712555047793, train_acc: 0.5765224358974357, valid_score: 0.28144144411065625, time: 17.418983936309814


acc 0.6953125
acc 0.6328125
acc 0.65625
acc 0.671875
acc 0.7578125
acc 0.609375
acc 0.59375
acc 0.640625
acc 0.671875
acc 0.6875
acc 0.7421875
acc 0.6640625
acc 0.546875
acc 0.625
acc 0.6796875
acc 0.625
acc 0.703125
acc 0.6796875
acc 0.640625
acc 0.6328125
acc 0.6796875
acc 0.6328125
acc 0.609375
acc 0.578125
acc 0.7109375
acc 0.75
acc 0.7109375
acc 0.6328125
acc 0.734375
acc 0.8046875
acc 0.7421875
acc 0.6171875
acc 0.671875
acc 0.6796875
acc 0.71875
acc 0.7265625
acc 0.671875
acc 0.7265625
acc 0.671875
acc 0.65625
acc 0.6328125
acc 0.625
acc 0.7265625
acc 0.7421875
acc 0.71875
acc 0.7109375
acc 0.7578125
acc 0.703125
acc 0.6640625
acc 0.7421875
acc 0.671875
acc 0.6796875
acc 0.7578125
acc 0.6875
acc 0.640625
acc 0.75
acc 0.6953125
acc 0.7578125
acc 0.71875
acc 0.71875
acc 0.7734375
acc 0.71875
acc 0.7109375
acc 0.65625
acc 0.640625
acc 0.7421875
acc 0.6640625
acc 0.6875
acc 0.609375
acc 0.6875
acc 0.7265625
acc 0.7265625
acc 0.6640625
acc 0.6796875
acc 0.765625
acc 0.6953125
acc 0.7

Epoch 2, train_loss: 0.5978239755599927, train_acc: 0.6871995192307694, valid_score: 0.2744143547698396, time: 17.757356882095337
INFO:main:Epoch 2, train_loss: 0.5978239755599927, train_acc: 0.6871995192307694, valid_score: 0.2744143547698396, time: 17.757356882095337


...

Epoch 5, train_loss: 0.5127294182013241, train_acc: 0.7451923076923078, valid_score: 0.2735738836535009, time: 17.735079526901245
INFO:main:Epoch 5, train_loss: 0.5127294182013241, train_acc: 0.7451923076923078, valid_score: 0.2735738836535009, time: 17.735079526901245


acc 0.796875
acc 0.7265625
acc 0.75
acc 0.7734375
acc 0.8046875
acc 0.7734375
acc 0.75
acc 0.8046875
acc 0.7890625
acc 0.8046875
acc 0.7265625
acc 0.796875
acc 0.78125
acc 0.8359375
acc 0.796875
acc 0.78125
acc 0.734375
acc 0.7734375
acc 0.7421875
acc 0.8203125
acc 0.7578125
acc 0.7890625
acc 0.75
acc 0.78125
acc 0.765625
acc 0.78125
acc 0.71875
acc 0.7578125
acc 0.7578125
acc 0.7578125
acc 0.7578125
acc 0.796875
acc 0.7890625
acc 0.8125
acc 0.703125
acc 0.78125
acc 0.7890625
acc 0.7890625
acc 0.7109375
acc 0.7265625
acc 0.7421875
acc 0.7421875
acc 0.7890625
acc 0.78125
acc 0.7421875
acc 0.765625
acc 0.703125
acc 0.7578125
acc 0.7265625
acc 0.7578125
acc 0.7578125
acc 0.78125
acc 0.71875
acc 0.7109375
acc 0.7421875
acc 0.765625
acc 0.7421875
acc 0.7734375
acc 0.8203125
acc 0.71875
acc 0.8046875
acc 0.7421875
acc 0.734375
acc 0.7265625
acc 0.828125
acc 0.765625
acc 0.828125
acc 0.75
acc 0.765625
acc 0.7265625
acc 0.78125
acc 0.75
acc 0.6875
acc 0.84375
acc 0.7109375
acc 0.6953125
acc 0.

Epoch 6, train_loss: 0.49001954992612207, train_acc: 0.7633213141025643, valid_score: 0.29583461147924506, time: 17.26571273803711
INFO:main:Epoch 6, train_loss: 0.49001954992612207, train_acc: 0.7633213141025643, valid_score: 0.29583461147924506, time: 17.26571273803711


acc 0.7734375
acc 0.65625
acc 0.8515625
acc 0.8125
acc 0.7578125
acc 0.8046875
acc 0.71875
acc 0.765625
acc 0.7890625
acc 0.7890625
acc 0.8046875
acc 0.8359375
acc 0.734375
acc 0.765625
acc 0.7578125
acc 0.734375
acc 0.7421875
acc 0.734375
acc 0.8125
acc 0.71875
acc 0.765625
acc 0.7734375
acc 0.7578125
acc 0.71875
acc 0.765625
acc 0.7734375
acc 0.734375
acc 0.7578125
acc 0.8125
acc 0.734375
acc 0.8125
acc 0.7421875
acc 0.7421875
acc 0.703125
acc 0.734375
acc 0.7734375
acc 0.8203125
acc 0.7890625
acc 0.828125
acc 0.75
acc 0.75
acc 0.7890625
acc 0.71875
acc 0.75
acc 0.8046875
acc 0.7578125
acc 0.7578125
acc 0.7109375
acc 0.7890625
acc 0.6953125
acc 0.8515625
acc 0.7578125
acc 0.796875
acc 0.7421875
acc 0.734375
acc 0.796875
acc 0.7578125
acc 0.8203125
acc 0.75
acc 0.7578125
acc 0.8203125
acc 0.7109375
acc 0.765625
acc 0.7421875
acc 0.7734375
acc 0.796875
acc 0.78125
acc 0.75
acc 0.78125
acc 0.7578125
acc 0.828125
acc 0.8359375
acc 0.734375
acc 0.7890625
acc 0.7265625
acc 0.7734375
acc 0.

Epoch 7, train_loss: 0.4833323374772681, train_acc: 0.7686298076923075, valid_score: 0.27772753152876256, time: 17.096177577972412
INFO:main:Epoch 7, train_loss: 0.4833323374772681, train_acc: 0.7686298076923075, valid_score: 0.27772753152876256, time: 17.096177577972412


## Inference

In [41]:
model = KDDModel().to(device)
model.load_state_dict(torch.load(MODEL_DIR+f'best_param.pt'))

NameError: ignored

In [0]:
def make_submit_df(df_test, model):
    dataset_test = KDDDataset(df_test, train_mode=False)
    test_loader = DataLoader(dataset_test, batch_size=256, shuffle=False, num_workers=0, drop_last=False)
    out_list = test_model(test_loader, model)

    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    submit_df = pd.DataFrame(columns=['query-id','product1','product2','product3','product4','product5'])
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        submit_df.loc[i, 'query-id'] = q_id
        submit_df.iloc[i, 1:] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values
    submit_df = submit_df.astype(int)
    submit_df = submit_df.sort_values(by='query-id')
    submit_df = submit_df.reset_index(drop=True)
    
    return submit_df

In [0]:
submit_df = make_submit_df(testA, model)
submit_df.to_csv(SUBMIT_DIR+'submission.csv', index=False)